# Football Match Outcome Prediction
Using Machine Learning to Predict European Soccer Match Results
**Dataset:** European Soccer Database (Kaggle)  
**Time Period:** 2008-2016  
**Matches:** 25,000+  
**Objective:** Predict Win/Draw/Loss outcomes using team attributes and betting odds

# Import Libraries and Setup


In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

import warnings
warnings.filterwarnings('ignore')

# Load Data from SQLite Database


In [2]:
# Database path
DB_PATH = r"database\database.sqlite"

def load_data(db_path):
    """Load all relevant tables from SQLite database"""
    conn = sqlite3.connect(db_path)
    
    # Load tables
    matches = pd.read_sql('SELECT * FROM "Match"', conn)
    teams = pd.read_sql('SELECT * FROM Team', conn)
    team_attributes = pd.read_sql('SELECT * FROM Team_Attributes', conn)
    
    conn.close()
    
    print(f"Successfully loaded:")
    print(f"   • {len(matches):,} matches")
    print(f"   • {len(teams):,} teams")
    print(f"   • {len(team_attributes):,} team attribute records")
    
    return matches, teams, team_attributes

# Load data
matches, teams, team_attributes = load_data(DB_PATH)

Successfully loaded:
   • 25,979 matches
   • 299 teams
   • 1,458 team attribute records
